In [36]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score,roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
#from analytics_tools.analytics_tools3 import freq_discrete
%matplotlib inline

In [2]:
df = pd.read_excel('Data/churn.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2801 entries, 0 to 2800
Data columns (total 14 columns):
V_ACCT_LGHT      2801 non-null int64
C_IP             2801 non-null object
C_VMP            2801 non-null object
V_N_VM_MESS      2801 non-null int64
V_T_DAY_CHG      2801 non-null float64
V_T_EVE_CALLS    2801 non-null int64
V_T_EVE_CHG      2801 non-null float64
V_T_NIG_CALLS    2801 non-null int64
V_T_NIG_CHG      2801 non-null float64
V_T_INT_CALLS    2801 non-null int64
V_T_INT_CHG      2801 non-null float64
V_CS_CALLS       2801 non-null int64
TARGET           2801 non-null int64
ID               2801 non-null int64
dtypes: float64(4), int64(8), object(2)
memory usage: 306.4+ KB


In [4]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,2.70,1,0,1
1,107,No,Yes,26,27.47,103,16.62,103,11.45,3,3.70,1,0,2
2,137,No,No,0,41.38,110,10.30,104,7.32,5,3.29,0,0,3
3,65,No,No,0,21.95,83,19.42,111,9.40,6,3.43,4,1,11
4,168,No,No,0,21.90,71,8.92,128,6.35,2,3.02,1,0,13


In [7]:
var_disc = [v for v in df.columns if v[:2]=='C_']
var_cont = [v for v in df.columns if v[:2]=='V_']

In [9]:
for v in var_disc:
    df[v] = (df[v]=='Yes').astype(int)

In [10]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,TARGET,ID
0,128,0,1,25,45.07,99,16.78,91,11.01,3,2.70,1,0,1
1,107,0,1,26,27.47,103,16.62,103,11.45,3,3.70,1,0,2
2,137,0,0,0,41.38,110,10.30,104,7.32,5,3.29,0,0,3
3,65,0,0,0,21.95,83,19.42,111,9.40,6,3.43,4,1,11
4,168,0,0,0,21.90,71,8.92,128,6.35,2,3.02,1,0,13


In [11]:
X = df[var_cont+var_disc].copy()
y = df['TARGET'].copy()

In [14]:
scaler = MinMaxScaler()
scaler.fit(X)
Xs = pd.DataFrame(scaler.transform(X),columns=X.columns)

In [15]:
Xs.head()

,V_ACCT_LGHT,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,C_IP,C_VMP
0,0.633880,0.581395,0.841885,0.489362,0.492640,0.377778,0.693383,0.166667,0.500000,0.166667,0.0,1.0
1,0.519126,0.604651,0.433248,0.531915,0.484789,0.511111,0.734390,0.166667,0.776243,0.166667,0.0,1.0
2,0.683060,0.000000,0.756211,0.606383,0.174681,0.522222,0.349487,0.333333,0.662983,0.000000,0.0,0.0
3,0.289617,0.000000,0.305085,0.319149,0.622179,0.600000,0.543336,0.416667,0.701657,0.666667,0.0,0.0
4,0.852459,0.000000,0.303924,0.191489,0.106968,0.788889,0.259087,0.083333,0.588398,0.166667,0.0,0.0


In [20]:
Xt,Xv,yt,yv = train_test_split(Xs,y,train_size=0.7)

In [22]:
model = MLPClassifier()

In [23]:
%%time
model.fit(Xt,yt)

CPU times: user 3.14 s, sys: 4.07 s, total: 7.21 s
Wall time: 1.86 s


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [24]:
def metricas(model,Xt,Xv,yt,yv):
    print "Accuracy_Train %.2f, Accuracy_Validate %.2f "%(accuracy_score(y_pred=model.predict(Xt),y_true=yt),accuracy_score(y_pred=model.predict(Xv),y_true=yv))
    print "ROC_Train %.2f, ROC_Validate %.2f "%(roc_auc_score(y_score=model.predict_proba(Xt)[:,1],y_true=yt),roc_auc_score(y_score=model.predict_proba(Xv)[:,1],y_true=yv))


In [25]:
metricas(model,Xt,Xv,yt,yv)

Accuracy_Train 0.92, Accuracy_Validate 0.91 
ROC_Train 0.91, ROC_Validate 0.90 


In [37]:
param_grid = dict( hidden_layer_sizes=[(a,b,c,) for a in range(24,31) for b in range(24,31) for c in range(24,31)],
                  activation = ['identity', 'logistic', 'tanh', 'relu'],
                  alpha = np.arange(0.0001,0.0006,0.0001),
                  learning_rate = ['constant', 'invscaling', 'adaptive'],
                  max_iter = range(2000,2100,100)
                  
                  
)

In [48]:
grid = RandomizedSearchCV(cv=3,
                          verbose=True,
                          param_distributions=param_grid,
                          estimator=model,
                          n_jobs=-1,
                          n_iter=10,
                          scoring='roc_auc')

In [49]:
%%time
grid.fit(Xs,y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.9s finished


CPU times: user 1.2 s, sys: 43.3 ms, total: 1.25 s
Wall time: 8.96 s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'alpha': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005]), 'learning_rate': ['constant', 'invscaling', 'adaptive'], 'max_iter': [2000], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'hidden_layer_sizes': [(24, 24, 24), (24, 24, 25), (24, 24, 26), (24, 24, 27), (24, 24, ..., (30, 30, 24), (30, 30, 25), (30, 30, 26), (30, 30, 27), (30, 30, 28), (30, 30, 29), (30, 30, 30)]},
          pre_dispatch='2*n

In [50]:
grid.best_estimator_

MLPClassifier(activation='relu', alpha=0.00030000000000000003,
       batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(26, 30, 27),
       learning_rate='constant', learning_rate_init=0.001, max_iter=2000,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [51]:
metricas(grid.best_estimator_,Xt,Xv,yt,yv)

Accuracy_Train 0.93, Accuracy_Validate 0.94 
ROC_Train 0.91, ROC_Validate 0.91 
